# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
weather_df=pd.read_csv('../output/cities.csv')

In [70]:
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Urucará,-2.5364,-57.7600,86.77,60,83,5.23,BR,1643732699
1,Muli,2.9167,73.5667,83.55,74,39,14.45,MV,1643732700
2,Yellowknife,62.4560,-114.3525,-27.58,75,0,9.22,CA,1643732456
3,Cabedelo,-6.9811,-34.8339,88.47,64,13,12.82,BR,1643732700
4,Union,40.6334,-74.2663,36.27,57,0,5.75,US,1643732436


In [71]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)


In [72]:
locations

,Lat,Lng
0,-2.5364,-57.7600
1,2.9167,73.5667
2,62.4560,-114.3525
3,-6.9811,-34.8339
4,40.6334,-74.2663
...,...,...
542,-30.0319,-70.7081
543,-16.9167,145.7667
544,34.6851,135.8049
545,46.5389,138.3297


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
weather_df.head(4)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Urucará,-2.5364,-57.7600,86.77,60,83,5.23,BR,1643732699
1,Muli,2.9167,73.5667,83.55,74,39,14.45,MV,1643732700
2,Yellowknife,62.4560,-114.3525,-27.58,75,0,9.22,CA,1643732456
3,Cabedelo,-6.9811,-34.8339,88.47,64,13,12.82,BR,1643732700


In [77]:
#3 condition:(A max temperature lower than 80 degrees but higher than 72.
#Wind speed less than 10 mph.
#Zero cloudiness.)
hotel_df=weather_df.loc[(weather_df['Max Temp']>72)&(weather_df['Max Temp']<80)&(weather_df['Wind Speed']<10)\
                            &(weather_df['Cloudiness']==0)]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Chui,-33.6971,-53.4616,74.86,38,0,3.91,UY,1643732789
197,Mormugao,15.4000,73.8000,78.06,78,0,5.75,IN,1643733080
247,Rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643733155
280,Salalah,17.0151,54.0924,75.29,60,0,4.61,OM,1643733224
355,Iquique,-20.2208,-70.1431,75.24,65,0,8.63,CL,1643733306
506,Bouza,14.4229,6.0428,75.20,9,0,7.61,NE,1643733590
534,‘Ewa Beach,21.3156,-158.0072,73.35,82,0,3.00,US,1643733598


In [78]:
hotel_df.assign(Hotel_name="")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name
23,Chui,-33.6971,-53.4616,74.86,38,0,3.91,UY,1643732789,
197,Mormugao,15.4000,73.8000,78.06,78,0,5.75,IN,1643733080,
247,Rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643733155,
280,Salalah,17.0151,54.0924,75.29,60,0,4.61,OM,1643733224,
355,Iquique,-20.2208,-70.1431,75.24,65,0,8.63,CL,1643733306,
506,Bouza,14.4229,6.0428,75.20,9,0,7.61,NE,1643733590,
534,‘Ewa Beach,21.3156,-158.0072,73.35,82,0,3.00,US,1643733598,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotel
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [80]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
23,Chui,-33.6971,-53.4616,74.86,38,0,3.91,UY,1643732789,Aquarella Hotel Resort
197,Mormugao,15.4000,73.8000,78.06,78,0,5.75,IN,1643733080,The Baga Beach Resort
247,Rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643733155,UY Proa Sur Hotel
280,Salalah,17.0151,54.0924,75.29,60,0,4.61,OM,1643733224,Hilton Salalah Resort
355,Iquique,-20.2208,-70.1431,75.24,65,0,8.63,CL,1643733306,Hotel Terrado Cavancha
506,Bouza,14.4229,6.0428,75.20,9,0,7.61,NE,1643733590,NaN
534,‘Ewa Beach,21.3156,-158.0072,73.35,82,0,3.00,US,1643733598,Halekulani Hotel


In [81]:
hotel_df.dropna(inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
23,Chui,-33.6971,-53.4616,74.86,38,0,3.91,UY,1643732789,Aquarella Hotel Resort
197,Mormugao,15.4000,73.8000,78.06,78,0,5.75,IN,1643733080,The Baga Beach Resort
247,Rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643733155,UY Proa Sur Hotel
280,Salalah,17.0151,54.0924,75.29,60,0,4.61,OM,1643733224,Hilton Salalah Resort
355,Iquique,-20.2208,-70.1431,75.24,65,0,8.63,CL,1643733306,Hotel Terrado Cavancha
534,‘Ewa Beach,21.3156,-158.0072,73.35,82,0,3.00,US,1643733598,Halekulani Hotel


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Save Data to csv
hotel_df.to_csv("../output/Hotel_Output.csv",index=False)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
23,Chui,-33.6971,-53.4616,74.86,38,0,3.91,UY,1643732789,Aquarella Hotel Resort
197,Mormugao,15.4000,73.8000,78.06,78,0,5.75,IN,1643733080,The Baga Beach Resort
247,Rocha,-34.4833,-54.3333,78.19,34,0,2.98,UY,1643733155,UY Proa Sur Hotel
280,Salalah,17.0151,54.0924,75.29,60,0,4.61,OM,1643733224,Hilton Salalah Resort
355,Iquique,-20.2208,-70.1431,75.24,65,0,8.63,CL,1643733306,Hotel Terrado Cavancha


In [84]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…